In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

#### O Timeline controla a sincronização de todos os eventos, enquanto os QuantumRouter e BSMNode representam os nós da rede capazes de criar e medir estados de Bell, respectivamente. Os QuantumChannel e ClassicalChannel facilitam a transmissão de informações quânticas e clássicas entre os nós, permitindo a implementação dos protocolos de emaranhamento e comunicação na rede

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

#### Este código realiza uma simulação de uma rede quântica utilizando um modelo baseado em roteadores quânticos, canais clássicos e canais quânticos

In [3]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    Função para testar a simulação de uma rede quântica composta por roteadores quânticos e nós de medição Bell.
    
    Parâmetros:
    sim_time: duração da simulação (em milissegundos)
    cc_delay: atraso nos canais clássicos (em milissegundos)
    qc_atten: atenuação nos canais quânticos (em dB/m)
    qc_dist: distância dos canais quânticos (em km)
    """
    
    PS_PER_MS = 1e9  # Constante para converter milissegundos para picosegundos
    M_PER_KM = 1e3   # Constante para converter quilômetros para metros
    
    # Converter unidades de delay (para ps) e distância (para m)
    cc_delay *= PS_PER_MS
    qc_dist *= M_PER_KM
    
    raw_fidelity = 0.85  # Fidelidade bruta inicial dos pares EPR
    
    # Criar a linha do tempo da simulação (em ps)
    tl = Timeline(sim_time * PS_PER_MS)
    
    ## Criar os elementos da rede quântica e ajustar seus parâmetros conforme necessário
    
    # Criar os roteadores quânticos (nome, timeline, número de memórias quânticas)
    r1 = QuantumRouter("r1", tl, 50)
    r2 = QuantumRouter("r2", tl, 100)
    r3 = QuantumRouter("r3", tl, 50)
    
    # Criar os nós de Medição Bell (BSM) (nome, timeline, roteadores conectados)
    m1 = BSMNode("m1", tl, ["r1", "r2"])
    m2 = BSMNode("m2", tl, ["r2", "r3"])
    
    # Adicionar nós de BSM aos roteadores conectados
    r1.add_bsm_node(m1.name, r2.name)
    r2.add_bsm_node(m1.name, r1.name)
    r2.add_bsm_node(m2.name, r3.name)
    r3.add_bsm_node(m2.name, r2.name)
    
    # Definir sementes para geradores aleatórios nos nós (garantir reprodutibilidade)
    nodes = [r1, r2, r3, m1, m2]
    for i, node in enumerate(nodes):
        node.set_seed(i)
    
    # Atualizar parâmetros das memórias quânticas (tempo de coerência e fidelidade)
    for node in [r1, r2, r3]:
        memory_array = node.get_components_by_type("MemoryArray")[0]
        # Atualizar tempo de coerência (em segundos)
        memory_array.update_memory_params("coherence_time", 10)
        # Atualizar fidelidade bruta dos emaranhamentos
        memory_array.update_memory_params("raw_fidelity", raw_fidelity)
    
    # Criar canais de comunicação clássicos (all-to-all)
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            # Criar canal de comunicação clássico (nome, timeline, comprimento em m, delay em ps)
            cc = ClassicalChannel("cc_%s_%s" % (node1.name, node2.name), tl, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2.name)
            
    # Criar canais quânticos entre r1/r2 e m1
    qc0 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
    qc1 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
    qc0.set_ends(r1, m1.name)
    qc1.set_ends(r2, m1.name)
    
    # Criar canais quânticos entre r2/r3 e m2
    qc2 = QuantumChannel("qc_r2_m2", tl, qc_atten, qc_dist)
    qc3 = QuantumChannel("qc_r3_m2", tl, qc_atten, qc_dist)
    qc2.set_ends(r2, m2.name)
    qc3.set_ends(r3, m2.name)

    # Criar tabela de roteamento manualmente
    # Adicionar regras de roteamento baseadas nos links quânticos
    r1.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
    r1.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r2")
    r2.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r1")
    r2.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r3")
    r3.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r2")
    r3.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
    
    ## Executar a simulação
    
    # Inicializar a linha do tempo
    tl.init()
    
    # Fazer uma solicitação de emaranhamento usando o gerenciador de rede do roteador `r1`
    # Parâmetros: (nome do nó de destino, início do tempo de emaranhamento, fim do tempo, número de memórias, fidelidade desejada)
    r1.network_manager.request("r3", 1e12, 1e14, 50, 0.9)

    # Executar a simulação e medir o tempo de execução
    tick = time.time()
    tl.run()
    print("Tempo de execução: %.2f segundos" % (time.time() - tick))
    
    ## Exibir métricas das memórias emaranhadas
    
    # Criar subplots para mostrar número de memórias emaranhadas ao longo do tempo para r1, r2, e r3
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(12, 5)

    # Memórias emaranhadas no roteador r1
    data = []
    for info in r1.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)  # Converter tempo para segundos
    data.sort()
    ax1.plot(data, range(1, len(data) + 1), marker="o")
    ax1.set_title("r1")
    ax1.set_ylabel("Número de Memórias Emaranhadas")
    
    # Memórias emaranhadas no roteador r2
    data = []
    for info in r2.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)  # Converter tempo para segundos
    data.sort()
    ax2.plot(data, range(1, len(data) + 1), marker="o")
    ax2.set_title("r2")
    ax2.set_xlabel("Tempo de Simulação (s)")
    
    # Memórias emaranhadas no roteador r3
    data = []
    for info in r3.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)  # Converter tempo para segundos
    data.sort()
    ax3.plot(data, range(1, len(data) + 1), marker="o")
    ax3.set_title("r3")
    
    fig.tight_layout()

    ## Exibir métricas de fidelidade das memórias
    
    # Criar subplots para mostrar a fidelidade das memórias em r1, r2, e r3
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(12, 5)
    
    # Fidelidade das memórias no roteador r1
    data = []
    for info in r1.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax1.bar(range(len(data)), data)
    ax1.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")  # Linha pontilhada para fidelidade inicial
    ax1.plot([0, len(data)], [0.9, 0.9], "k--")  # Linha pontilhada para fidelidade desejada
    ax1.set_ylim(0.7, 1)
    ax1.set_title("r1")
    ax1.set_ylabel("Fidelidade")

    # Fidelidade das memórias no roteador r2
    data = []
    for info in r2.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax2.bar(range(len(data)), data)
    ax2.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")  # Linha pontilhada para fidelidade inicial
    ax2.plot([0, len(data)], [0.9, 0.9], "k--")  # Linha pontilhada para fidelidade desejada
    ax2.set_ylim(0.7, 1)
    ax2.set_title("r2")
    ax2.set_xlabel("Número da Memória")

    # Fidelidade das memórias no roteador r3
    data = []
    for info in r3.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax3.bar(range(len(data)), data)
    ax3.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")  # Linha pontilhada para fidelidade inicial
    ax3.plot([0, len(data)], [0.9, 0.9], "k--")  # Linha pontilhada para fidelidade desejada
    ax3.set_ylim(0.7, 1)
    ax3.set_title("r3")
    
    fig.tight_layout()

#### Criação de um widget interativo para visualizar a simulação com diferentes parâmetros

In [4]:
interactive_plot = interact(test, 
                            sim_time=(2000, 4000, 500),  # Tempo de simulação em milissegundos: valores variando de 2000 a 4000, com incrementos de 500 ms
                            cc_delay=(0.1, 1, 0.1),  # Atraso no canal clássico: variando de 0.1 a 1 ms, com incrementos de 0.1 ms
                            qc_atten=[1e-5, 2e-5, 3e-5],  # Atenuação no canal quântico: lista de três valores possíveis [1e-5, 2e-5, 3e-5]
                            qc_dist=(1, 10, 1))  # Distância do canal quântico: variando de 1 a 10 km, com incrementos de 1 km

# interactive_plot agora é um widget que permite ajustar os parâmetros e observar os resultados da função test()


interactive(children=(IntSlider(value=3000, description='sim_time', max=4000, min=2000, step=500), FloatSlider…